In [ ]:
# imports for conputer vision API (OpenCV, Mediapipe)
import cv2
import mediapipe as mp
import time

# imports for Spotipy API
import spotipy
import json
import webbrowser
import pandas as pd
import spotipy.util as util

In [ ]:
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [ ]:
import os
os.sys.path
# checking if I am using the right version of Python
# if it's 310 - it's good

In [ ]:
# open file with keys and set the path to your credentials JSON file
credentials = "gitignore/spotify_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [ ]:
# reading the keys and assign each to a variable
# A redirect URI, or reply URL, is the location where the authorization server sends the user once the app has been 
# successfully authorized and granted an authorization code or access token.
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [ ]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-read-currently-playing user-modify-playback-state app-remote-control streaming'
# streaming - might need this but this needs premium account
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)
# this creates a cache file in the folder that saves the token
# you also have to .gitignore this

In [ ]:
token
# make sure to clear the output on this cell

In [ ]:
# Create Spotify Object
sp = spotipy.Spotify(auth=token)

In [ ]:
webbrowser.open('https://open.spotify.com/')
# Also open your spotify on your mobile webbrowser too
# please open the computer browser first and then the mobile

In [ ]:
# Before running this code, make sure to open spotify on your device browser, in order to get detected
# Also make sure to refresh it if you are already there

# Make sure to play a song in your browser to refresh

devices = sp.devices()
print(devices)

In [ ]:
# Assigning device id into var
# Sometimes there are random unrecognized devices, so I get the most recent 2 devices.
deviceID_Mobile = devices['devices'][-1]['id']
deviceID_Web = devices['devices'][-2]['id']

print(deviceID_Mobile)
print(deviceID_Web)

In [ ]:
devices['devices'][-1]['is_active'] = True

In [ ]:
devices['devices'][-1]['is_active']

In [ ]:
###############################################################################################
# Before you run this line, make sure that you play a music in spotify that you want to conduct

# stop playback
sp.pause_playback()

In [ ]:
# Resume playback
sp.start_playback()

In [ ]:
song_play = sp.current_playback() #get playback status
print(song_play)

In [ ]:
if_playing = song_play["is_playing"] #get playback status
print(if_playing)

In [ ]:
sp.transfer_playback(deviceID_Mobile)

In [ ]:
sp.transfer_playback(deviceID_Web)

In [ ]:
# this takes sometime

mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

cap = cv2.VideoCapture(0)

# cap = cv2.flip(src, 0)
## flipping the video ##

spotify_url = 'https://open.spotify.com/'

on_left = False
last_on_left_time = time.time()

In [ ]:
#Chcking the video size
video_width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
video_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`
print(video_width, video_height)

In [ ]:
while True:
    ret, frame = cap.read()
    # ret is a boolean variable that returns true if the frame is available.
    # frame is an image array vector captured based on the default frames per second defined explicitly or implicitly

    if not ret: 
        break
        # if there is no frame avaiable, escape from the loop. if it says continue, continue with the loop

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) 
    # changing the default BRG colour to RGB

    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:

            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            # tracking index finger

            #######################################################
            # GOAL1 -> map the x value to adjust the device playing
            #######################################################

            if index_tip.x < 0.5 and on_left == False: 
            # see if the x position of the index finger is on the left side of the camera
            # x position value is 0-1
            # the second part checks if your hand is already on the left side
                
                sp.transfer_playback(deviceID_Mobile)
                on_left = True
                # change on_left value to be True
            
            elif index_tip.x >= 0.5 and on_left == True:
                sp.transfer_playback(deviceID_Web)
                on_left = False
                sp.volume(int(index_tip.y* 100))

            ###############################################
            # GOAL2 -> map the y value to adjust the volume
            # But only do this when the device is web
            # Cuz mobile it does not work
            ###############################################

            if index_tip.y < 0.1 and on_left == False:
                sp.volume(100)
            
            elif index_tip.y >= 0.1 and index_tip.y < 0.2 and on_left == False:
                sp.volume(90)

            elif index_tip.y >= 0.2 and index_tip.y < 0.3 and on_left == False:
                sp.volume(80)

            elif index_tip.y >= 0.3 and index_tip.y < 0.4 and on_left == False:
                sp.volume(70)

            elif index_tip.y >= 0.4 and index_tip.y < 0.5 and on_left == False:
                sp.volume(60)

            elif index_tip.y >= 0.5 and index_tip.y < 0.6 and on_left == False:
                sp.volume(50)

            elif index_tip.y >= 0.6 and index_tip.y < 0.7 and on_left == False:
                sp.volume(40)

            elif index_tip.y >= 0.7 and index_tip.y < 0.8 and on_left == False:
                sp.volume(30)

            elif index_tip.y >= 0.8 and index_tip.y < 0.9 and on_left == False:
                sp.volume(20)
                            
            elif index_tip.y >= 0.9 and index_tip.y < 1 and on_left == False:
                sp.volume(10)

            # volume must be and int
            

            # drawing a circle on each landmark(tracking points)
            for landmark in hand_landmarks.landmark:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
                # cv2.circle(image, center_coordinates, radius, color, thickness)


    # cv2.putText(frame, f'Touches: {touch_count}/3', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
sp.volume(0, device_id=deviceID_01)